In [9]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm


In [11]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [12]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label,"completed")
    return image_paths,labels
            

In [13]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [14]:
print(train)

                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [15]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [16]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale = True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features
    

In [17]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

H:\emomagnet\emotion detection\trail-1\myenv\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [18]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [10]:
x_train = train_features/255.0
x_test = test_features/255.0

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [13]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [14]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [15]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256,kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
#outputLayer
model.add(Dense(7, activation='softmax'))


In [16]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy')

In [17]:
model.fit(x=x_train,y=y_train,batch_size=128,epochs=100,validation_data=(x_test,y_test))

Epoch 1/100


226/226 [==============================] - 196s 862ms/step - loss: 1.8182 - accuracy: 0.2447 - val_loss: 1.7674 - val_accuracy: 0.2576
Epoch 2/100
226/226 [==============================] - 200s 887ms/step - loss: 1.6703 - accuracy: 0.3326 - val_loss: 1.5631 - val_accuracy: 0.3890
Epoch 3/100
226/226 [==============================] - 208s 923ms/step - loss: 1.4984 - accuracy: 0.4191 - val_loss: 1.3673 - val_accuracy: 0.4696
Epoch 4/100
226/226 [==============================] - 218s 963ms/step - loss: 1.4102 - accuracy: 0.4555 - val_loss: 1.3339 - val_accuracy: 0.4912
Epoch 5/100
226/226 [==============================] - 241s 1s/step - loss: 1.3418 - accuracy: 0.4841 - val_loss: 1.2396 - val_accuracy: 0.5236
Epoch 6/100
226/226 [==============================] - 247s 1s/step - loss: 1.2980 - accuracy: 0.5053 - val_loss: 1.2090 - val_accuracy: 0.5430
Epoch 7/100
226/226 [==============================] - 243s 1s/step - loss: 1.2569 - accuracy: 0.5188 - val_loss: 1.1797 -

KeyboardInterrupt: 

In [18]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

H:\emomagnet\emotion detection\trail-1\myenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
from keras.models import model_from_json

In [5]:
json_file = open('emotiondetector.json','r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("emotiondetector.h5")

In [6]:
label = ['angry', 'disgust', 'fear', 'happy', 'neural', 'sad', 'surprise']

In [7]:
def ef(image):
    img = load_img(image,grayscale = True)
    feature = np.array(img)
    feature.reshape(1,48,48,1)
    return feature/255.0

In [10]:
image = 'images/train/angry/22.jpg'
print("original  image is of angry")
image = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ", pred_label)

original  image is of angry


H:\emomagnet\emotion detection\trail-1\myenv\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


NameError: name 'img' is not defined

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
image